# Derivados Financieros 2023 - [material previo a Clase 3 1/2]

## Menu del dia:

    - (1) Web scraping de pginas de opciones
    - (2) Limpieza y generacion de dataframe ad-hoc para futuros usos
    - (3) pyhomebroker (requiere acceso a ALyC que use HomeBroker) - NO SOPORTADO 2023
    - (4) algunos plots de market data de opciones
    - (5) exportar a exel/csv 
    - (6) Comparar paneles (rava-hb)  - NO SOPORTADO 2023
    
    

## (1) Web Scraping de la pagina de IOL


Modulos necesarios

data_byma (en webcampus) ;

utils_opciones_byma (en webcampus)

bs4,

requests,

pandas,

matplotlib,

datetime,

calendar,

sys,

Instalarlos con Anaconda, comando Conda o comando pip install

### Importo los modulos

In [ ]:

import sys

sys.path.append('..')
from Codigo.data_byma import *
from Codigo.utils_opciones_byma import *



### Metodo para el web scraping

In [ ]:
panel_opciones = web_scraping_opciones_iol()
panel_opciones.head(20)

### Que hace internamente el metodo??

#### Beautiful Soup /requests 


Mas sobre BS4: https://j2logo.com/python/web-scraping-con-python-guia-inicio-beautifulsoup/

Mas sobre requests: https://www.digitalocean.com/community/tutorials/how-to-get-started-with-the-requests-library-in-python-es

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#### Como usamos BeautifulSoup y requests

In [ ]:
#url = 'http://www.rava.com/precios/panel.php?m=OPC'
#url = 'http://clasico.rava.com/precios/panel.php?m=OPC'
#url = 'http://datos.rava.com/cotizaciones/opciones'
url = 'https://iol.invertironline.com/mercado/cotizaciones/argentina/opciones/todas'


page = requests.get(url)
page

In [ ]:
# En soup voy a encontrar el codigo html de la pagina

soup = BeautifulSoup(page.text, 'lxml')
soup

#### El arte del scraping

Aqui es donde uno tiene que meterse en el codigo interno de la pagina e 'identificar' la informacion que necesita. En este caso la tabla de informacion de opciones
Se puede hacer una inspeccion en modo dev del navegador


In [ ]:
# En opciones tengo el contenido de la tabla correspondiente

opciones = soup.find('table', {'id': 'cotizaciones'})
opciones

#### Aqui continua el detalle. Se identifican las filas de la tabla. *tr* sera el separador que va empezando y cerrando las filas

In [ ]:
# Filas ya va ordenando cada una de las opciones


filas = opciones.find_all('tr')
filas

#### Identifico la primera linea, la de los -headers- de la tabla

In [ ]:
tr = filas[0]
tr

#### Identifico cada columna dentro de la fila con el marcador td


In [ ]:
td = tr.find_all('td')
td


#### generamos una lista (ya un terreno mas conocido :)


In [ ]:
row = [tr.text for tr in td]
row

#### Proceso de ir almacenando las filas en una lista d elistas (eventualmente terminara siendo un dataframe)

In [ ]:
# inicializo una lista vacia, alli voy a ir poniendo el contenido de cada fila
l = []
l.append(row)

l

#### fila por fila  

In [ ]:


tr = filas[1]
tr

In [ ]:
td = tr.find_all('td')
row = [tr.text for tr in td]
row

####  Aca las primeras intervenciones sobre la data que viene del html para que sea mas facil manipularla

In [ ]:
#Reemplazo ',' por '.' y paso el string a float
for index in [1,2,3,4,5,6,8,9]:
    row[index] = row[index].replace('.', '')
    row[index] = row[index].replace(',', '.')
    try:
        row[index] = float(row[index])
    except:
        #Si no encuentra valor pone -99.99
        row[index] = -99.99

l.append(row)
    
l
    
    

In [ ]:
#Ahora si, todas las filas
for tr in filas[2:]:
    td = tr.find_all('td')
    row = [tr.text for tr in td]

    #Reemplazo ',' por '.' y paso el string a float
    for index in [1,2,3,4,5,6,8,9]:
        row[index] = row[index].replace('.', '')
        row[index] = row[index].replace(',', '.')
        try:
            row[index] = float(row[index])
        except:
            #Si no encuentra valor pone -99.99
            row[index] = -99.99

    l.append(row)

In [ ]:
# Como se ve l despues del ciclo?
l

#### Lo convierto en un DataFrame

In [ ]:
panel = pd.DataFrame(l[1:], columns=l[0])

panel




    ### lo mandamos a un csv

In [ ]:
#panel.to_csv('panel_opciones_iol.csv')

## Panel de acciones


In [ ]:
web_scraping_acciones_iol()

## (2) Del panel de Iol crudo a otro con informacion customizada

### Metodo para generar el panel customizado

a) Panel crudo mas informacion masticada (columnas de la derecha)

In [ ]:
panel_opciones_byma_all_raw = obtener_panel_opciones_byma(ticker = '', clean_flag = False)
panel_opciones_byma_all_raw

b) Panel limpiotodos los tickers

In [ ]:
panel_opciones_byma_all_clean = obtener_panel_opciones_byma(ticker = '', clean_flag = True)
panel_opciones_byma_all_clean

b) Panel limpio paa un ticker en particular

In [ ]:
panel_opciones_byma_GGAL_clean = obtener_panel_opciones_byma('GGAL', True)
panel_opciones_byma_GGAL_clean

### Que hace el metodo internamente?



#### 1) Levanta los paneles de opciones y de acciones y genera una copia del de opciones

In [ ]:
panel_rava = web_scraping_opciones_iol()
#Obtengo el panel crudo de acciones (para el spot)
panel_acciones = web_scraping_acciones_iol()

#Genero el nuevo panel
panel_opciones = panel_rava.copy()


#### 2) Inicializo las columnas nuevas a agregar del Dataframe

In [ ]:
#Le agrego las nuevas columnas (ojo con los tipos string, float, int, datetime)
panel_opciones['Ticker_Opcion'] = ''
panel_opciones['Ticker_Stock'] = ''
panel_opciones['Tipo'] = ''
panel_opciones['Strike'] = 0.0
panel_opciones['ExpiryMonthName'] = ''
panel_opciones['ExpiryMonthNumber'] = 0
panel_opciones['ExpiryDate'] = datetime.date.today()
panel_opciones['Spot'] = 0.0
panel_opciones['TTM'] = 0
panel_opciones['Moneyness'] = 0.0

panel_opciones

#### 3) Recorro fila por fila para llenar las nuevas columnas (1 ejemplo)

##### *) Desgloso la especie en ticker, strike, vencimiento y tipo

In [ ]:
fila = 0


panel_opciones.Ticker_Opcion.values[fila] = panel_opciones.Símbolo.values[fila][0:3]

panel_opciones.Tipo.values[fila] = panel_opciones.Símbolo.values[fila][3:4]
panel_opciones.Strike.values[fila] = panel_opciones.Símbolo.values[fila][4:8]
panel_opciones.ExpiryMonthName.values[fila] = panel_opciones.Símbolo.values[fila][8:10]

panel_opciones.head(2)

##### *) Uno a uno voy generando lo necesario

In [ ]:
panel_opciones.Ticker_Stock.values[fila] = conversor_ticker(panel_opciones.Ticker_Opcion.values[fila])

panel_opciones.Tipo.values[fila] = panel_opciones.Tipo.values[fila].replace('V', 'P')

panel_opciones.head(2)

In [ ]:
# Artilugio que lo tuve que usar NO RECUERDO POR QUE! :)

try:
    int(panel_opciones.ExpiryMonthName.values[fila][0])
    panel_opciones.Strike.values[fila] = float(panel_opciones.Strike.values[fila] + int(panel_opciones.ExpiryMonthName.values[fila][0]))
    panel_opciones.ExpiryMonthName.values[fila] = panel_opciones.ExpiryMonthName.values[fila][1]
except:
    panel_opciones.Strike.values[fila] = float(panel_opciones.Strike.values[fila])
    
    

In [ ]:
#conversor_ticker('AGR')

#panel_opciones.Especie.values[fila][0:3]

#panel_opciones.Ticker_Opcion.values[fila]\

#conversor_ticker(panel_opciones.Ticker_Opcion.values[fila])

#panel_opciones.Ticker_Stock.values[fila]



##### *) Obtengo el precio spot del ticker con un metodo interno y aprovechando el panel de acciones

In [ ]:
panel_opciones.Spot.values[fila] = obtener_spot_ticker(panel_acciones, panel_opciones.Ticker_Stock.values[fila])

##### *) Obtengo el Time to maturity (en dias)

In [ ]:
try:
    panel_opciones.ExpiryMonthNumber.values[fila] = mes_nombre_a_numero(panel_opciones.ExpiryMonthName.values[fila])
except:
    panel_opciones.ExpiryMonthNumber.values[fila] = 0

panel_opciones.ExpiryDate.values[fila] = fecha_expiracion(panel_opciones.ExpiryMonthNumber.values[fila])
hoy = datetime.date.today()
panel_opciones.TTM.values[fila] = (panel_opciones.ExpiryDate.values[fila] - hoy).days

panel_opciones

##### *) Por ultimo calculo el moneyness de valor informativo importante

In [ ]:
panel_opciones.Moneyness.values[fila] = panel_opciones.Spot.values[fila] / panel_opciones.Strike.values[fila]

In [ ]:
panel_opciones.head(2)

#### 3bis) Recorro fila por fila para llenar las nuevas columnas


In [ ]:

for fila in range(1,len(panel_opciones.Símbolo.values)):
    panel_opciones.Ticker_Opcion.values[fila] = panel_opciones.Símbolo.values[fila][0:3]

    panel_opciones.Ticker_Stock.values[fila] = conversor_ticker(panel_opciones.Ticker_Opcion.values[fila])

    panel_opciones.Tipo.values[fila] = panel_opciones.Símbolo.values[fila][3:4]
    panel_opciones.Tipo.values[fila] = panel_opciones.Tipo.values[fila].replace('V', 'P')

    panel_opciones.Strike.values[fila] = panel_opciones.Símbolo.values[fila][4:8]

    panel_opciones.ExpiryMonthName.values[fila] = panel_opciones.Símbolo.values[fila][8:10]

    try:
        int(panel_opciones.ExpiryMonthName.values[fila][0])
        panel_opciones.Strike.values[fila] = float(panel_opciones.Strike.values[fila] + int(panel_opciones.ExpiryMonthName.values[fila][0]))
        panel_opciones.ExpiryMonthName.values[fila] = panel_opciones.ExpiryMonthName.values[fila][1]
    except:
        panel_opciones.Strike.values[fila] = float(panel_opciones.Strike.values[fila])

    try:
        panel_opciones.ExpiryMonthNumber.values[fila] = mes_nombre_a_numero(panel_opciones.ExpiryMonthName.values[fila])
    except:
        panel_opciones.ExpiryMonthNumber.values[fila] = 0

    panel_opciones.ExpiryDate.values[fila] = fecha_expiracion(panel_opciones.ExpiryMonthNumber.values[fila])


    panel_opciones.Spot.values[fila] = obtener_spot_ticker(panel_acciones, panel_opciones.Ticker_Stock.values[fila])


    hoy = datetime.date.today()
    panel_opciones.TTM.values[fila] = (panel_opciones.ExpiryDate.values[fila] - hoy).days

    panel_opciones.Moneyness.values[fila] = panel_opciones.Spot.values[fila] / panel_opciones.Strike.values[fila]



In [ ]:
#Asi queda entonces el panel
panel_opciones

#### 4) Filtro por ticker


In [ ]:
#Si se provee un ticker, se filtra por ticker, si no, se provee el panel completo

ticker = 'ALUA'

if ticker == '':
    pass
else:
    panel_opciones = panel_opciones[panel_opciones.Ticker_Stock == ticker]
    
panel_opciones

#### 5) Panel limpio

In [ ]:
panel_limpio = panel_opciones.copy()

clean_flag = True

# Panel limpio: cambio de nombre, reordenamiento y limpieza minima del panel
if clean_flag == True:
    # Sacar opciones sobre bonos
    panel_limpio = panel_limpio[~panel_limpio.Símbolo.str.contains('A24')]

    panel_limpio = panel_limpio[
            ['Símbolo', 'Ticker_Stock', 'Spot', 'Tipo', 'Strike', 'TTM', 'ÚltimoOperado', 'Moneyness']]

    panel_limpio = panel_limpio.rename(
            columns={'Ticker_Stock': 'Ticker', 'Tipo': 'CallPut', 'ÚltimoOperado': 'ÚltimoCierre'}, inplace=False)

    panel_limpio = panel_limpio.sort_values(['TTM', 'Ticker', 'Strike', 'CallPut'])

    # Remover TTMs mayores de un año o negativos (por algun error)
    panel_limpio = panel_limpio[panel_limpio.TTM < 364]
    panel_limpio = panel_limpio[panel_limpio.TTM > 0]

    panel_limpio = panel_limpio.reset_index()
    del panel_limpio['index']
    
panel_limpio

## (3) pyhomebroker [No esta actualizado el codigo en 2023, puede fallar - revisar]


### pyhomebroker es una a API para conectar a un programa/script/notebook de python al mercado a traves de AlyCs que usen la plataforma homebroker (Estudio Gallo). Se usa para recibir precios y datos de mercado en tiempo real. Permite tambien obtener data historica.


Mas info e instrucciones de instalacion en https://pypi.org/project/pyhomebroker/

Estudio Gallo: http://www.estudiogallo.com.ar/

Home Broker: https://www.youtube.com/watch?v=ippFga_hKSg

### por obvias razones, no revelo mis datos personales :) Pero aqui habria que crear un diccionario con sus datos

In [ ]:
from data_byma_homebroker import *
import pickle
#TO SAVE
#personal_data = {'ByMA_id' : BymaId,
#                 'dni'     : dni, 
#                 'user'    : user,
#                 'password': password}
#pickle.dump(personal_data, open("personal_data.info", "wb"))

#TO LOAD
personal_data = pickle.load(open("personal_data.info", "rb"))




### Metodos disponibles

In [ ]:
panel_opciones_homebroker_all_raw = panel_opciones_homebroker(personal_data,
                                                              ticker = '', 
                                                              clean_flag=False)
panel_opciones_homebroker_all_raw


In [ ]:
panel_opciones_homebroker_all_clean = panel_opciones_homebroker(personal_data, 
                                                                ticker = '', 
                                                                clean_flag=True)
panel_opciones_homebroker_all_clean

In [ ]:
panel_opciones_homebroker_GGAL_clean = panel_opciones_homebroker(personal_data, 'GGAL', True)
panel_opciones_homebroker_GGAL_clean

### Que hace el metodo internamente?


#### a) Recupero la informacion del picke (esto es personal!)

In [ ]:
# Recupero la informacion del pickle

ByMA_id = personal_data['ByMA_id']
dni = personal_data['dni']
user = personal_data['user']
password = personal_data['password']

#### b) Conecto con el cliente

In [ ]:
hb = HomeBroker(ByMA_id)
hb.auth.login(dni, user, password, raise_exception=True)
# Connect to the server
hb.online.connect()

#### c) Obtengo una 'foto'-snapshot del mercado [diccionario]

In [ ]:
snapshot = hb.online.get_market_snapshot()
snapshot

In [ ]:
panel_opciones = snapshot['options']
panel_merval = snapshot['bluechips']
panel_general = snapshot['general_board']
panel_cedears = snapshot['cedears']



In [ ]:
panel_cedears


In [ ]:
panel_opciones

#### d) Limpiando el dataset

In [ ]:
# Le agrego columnas nuevas
panel_opciones['Spot'] = 0.0 
panel_opciones['TTM']=0
panel_opciones['Moneyness']=0.0

In [ ]:
#Genero la columna TTM
hoy = datetime.datetime.today()
      
#Codigo para obtener el TTM
x= (panel_opciones['expiration'] - hoy).values        
days = x.astype('timedelta64[D]')
ttm_f = (days+1) / np.timedelta64(1, 'D')
       
panel_opciones['TTM'] = ttm_f.astype(np.int64)

panel_opciones

In [ ]:
# Renombro las columnas, renombro el tipo Call/Put, reordeno y me quedo solo con las columnas que quiero
panel_opciones = panel_opciones.reset_index()
panel_opciones = panel_opciones.rename(
            columns={'symbol':'Especie','underlying_asset':'Ticker', 'kind': 'CallPut', 'last': 'Last', 'strike': 'Strike','previous_close':'Close'}, inplace=False)
       
panel_opciones['CallPut'] = pd.Series(panel_opciones['CallPut']).str.replace('CALL', 'C', regex=True)
panel_opciones['CallPut'] = pd.Series(panel_opciones['CallPut']).str.replace('PUT', 'P', regex=True)
       
panel_opciones = panel_opciones.sort_values(['TTM', 'Ticker', 'Strike', 'CallPut'])
panel_opciones = panel_opciones[
            ['Especie','Ticker', 'Spot', 'CallPut', 'Strike', 'TTM', 'Last', 'Close','Moneyness']]

In [ ]:
#Limpieza final
#Saco las Nan
panel_opciones = panel_opciones[pd.notna(panel_opciones.Last)]
# saco la columna indice vieja
panel_opciones = panel_opciones.reset_index()
del panel_opciones['index']


In [ ]:
# Agrego el spot y el moneyness
# get_spot_from_ticker es un metodo propio

for fila in range(len(panel_opciones.Especie.values)):
    ticker_ = panel_opciones.Ticker.values[fila]
    panel_opciones['Spot'].values[fila] = get_spot_from_ticker(ticker_,snapshot)
        
panel_opciones['Moneyness']= panel_opciones['Spot'] / panel_opciones['Strike']

panel_opciones

In [ ]:
#Limpieza final
#Saco las Nan
panel_opciones = panel_opciones[pd.notna(panel_opciones.Last)]
# saco la columna indice vieja
panel_opciones = panel_opciones.reset_index()
del panel_opciones['index']

In [ ]:
panel_opciones

In [ ]:
# SI solamente quiero un ticker

panel_opciones_ggal = panel_opciones[panel_opciones.Ticker == 'GGAL']

panel_opciones_ggal = panel_opciones_ggal.reset_index()
del panel_opciones_ggal['index']
panel_opciones_ggal

## (4) Ploteando los datos

### Para cada vencimiento, grafico los strikes

In [ ]:
# BYMA


panel_opciones_ggal_byma_limpio = obtener_panel_opciones_byma('GGAL', True)

TTMs = list(set(panel_opciones_ggal_byma_limpio.TTM.values))
TTMs

In [ ]:
# El vencimiento mas liquido
TTM = TTMs[1]

panel_opciones_ggal_byma_limpio[(panel_opciones_ggal_byma_limpio.TTM == TTM)].head(20)

In [ ]:
from matplotlib import pyplot as plt
CP = "C"
for TTM in TTMs:
    Ks = panel_opciones_ggal_byma_limpio[(panel_opciones_ggal_byma_limpio.TTM == TTM) & (panel_opciones_ggal_byma_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_ggal_byma_limpio[(panel_opciones_ggal_byma_limpio.TTM == TTM) & (panel_opciones_ggal_byma_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios,'*', label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()
    

In [ ]:

CP = "P"
for TTM in TTMs:
    Ks = panel_opciones_ggal_byma_limpio[(panel_opciones_ggal_byma_limpio.TTM == TTM) & (panel_opciones_ggal_byma_limpio.CallPut == CP)].Strike.values
    precios = panel_opciones_ggal_byma_limpio[(panel_opciones_ggal_byma_limpio.TTM == TTM) & (panel_opciones_ggal_byma_limpio.CallPut == CP)].Last.values
    plt.plot(Ks, precios,'*', label='TTM = '+str(TTM))


plt.legend()

plt.xlabel('Strike')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al Strike')
    
plt.show()

In [ ]:
Ks = list(set(panel_opciones_ggal_byma_limpio.Strike.values))

Ks[0:2]

In [ ]:

CP = "C"
for K in Ks:
    TTMs = panel_opciones_ggal_byma_limpio[(panel_opciones_ggal_byma_limpio.Strike == K) & (panel_opciones_ggal_byma_limpio.CallPut == CP)].TTM.values
    precios = panel_opciones_ggal_byma_limpio[(panel_opciones_ggal_byma_limpio.Strike == K) & (panel_opciones_ggal_byma_limpio.CallPut == CP)].Last.values
    if len(TTMs)>1:
        plt.plot(TTMs, precios,'*', label='Strike = '+str(K))

plt.legend()

plt.xlabel('TTM')
plt.ylabel('Precio de mercado del Call')
plt.title('Precio de mercado de las opciones Call con respecto al TTM')
    
plt.show()
panel_opciones_ggal_byma_limpio[(panel_opciones_ggal_byma_limpio.Strike == K) & (panel_opciones_ggal_byma_limpio.CallPut == CP)].Last.values
    

## (5) Pasar a Excel/CSV

In [ ]:
panel_opciones_byma_crudo = web_scraping_opciones_rava()
panel_opciones_byma_crudo.to_excel('Clase_1_panel_opciones_byma_crudo.xls')

panel_acciones_byma = obtener_panel_acciones()
panel_acciones_byma.to_excel('Clase_1_panel_acciones.xls')

panel_opciones_byma = obtener_panel_opciones_byma('', False)
panel_opciones_byma.to_excel('Clase_1_panel_opciones_byma.xls')

panel_opciones_byma_limpio = obtener_panel_opciones_byma('', True)
panel_opciones_byma_limpio.to_excel('Clase_1_panel_opciones_byma_limpio.xls')
panel_opciones_byma_limpio = obtener_panel_opciones_byma('', True)
panel_opciones_byma.to_excel('Clase_1_panel_opciones_byma.xls')
panel_opciones_byma_limpio.to_excel('Clase_1_panel_opciones_byma_limpio.xls')
                                                  
                                                  

## (6) Comparando los paneles

In [ ]:
panel_opciones_ggal_rava_limpio = obtener_panel_opciones_byma('GGAL',True)

In [ ]:
panel_opciones_ggal_homebroker_limpio = panel_opciones_homebroker(personal_data, 'GGAL', True)

In [ ]:
panel_opciones_ggal_rava_limpio

In [ ]:
panel_opciones_ggal_homebroker_limpio

In [ ]:
# Inner es la interseccion
# Outer es la union

merged = pd.merge(panel_opciones_ggal_rava_limpio, panel_opciones_ggal_homebroker_limpio, on=["Especie", "Ticker", "CallPut", "Strike", "TTM"], how='inner')

merged = merged.rename(
            columns={'Spot_x':'Spot_rava','Spot_y':'Spot_HB', 'Last_x': 'Last_rava', 'Last_y': 'Last_HB', 'Close_x': 'Close_rava', 'Close_y': 'Close_HB','previous_close':'Close'}, inplace=False)
       
merged = merged[
            ['Especie','Ticker', 'Spot_rava', 'Spot_HB', 'CallPut', 'Strike', 'TTM', 'Last_rava', 'Last_HB','Close_rava','Close_HB']]


merged

Genero algunas columnas de diferencias

In [ ]:

merged['Dif_spot']=merged['Spot_rava']-merged['Spot_HB']
merged['Dif_last']=merged['Last_rava']-merged['Last_HB']
merged['Dif_close']=merged['Close_rava']-merged['Close_HB']

merged

